# Implementing Lemke-Howson in Python: Hints

In [1]:
import numpy as np

In [2]:
np.set_printoptions(precision=5)  # Reduce the number of digits printed

In [3]:
A = np.array([[3, 3],
              [2, 5],
              [0 ,6]])
B_T = np.array([[3, 2, 3],
                [2, 6, 1]])

m, n = A.shape  # Numbers of actions of the players

To be consistent with the 0-based indexing in Python,
we call the players 0 and 1.

## Complementary pivoting

Build the tableau for each player:

In [4]:
# Player 0
tableau0 = np.empty((n, m+n+1))
tableau0[:, :m] = B_T
tableau0[:, m:m+n] = np.identity(n)
tableau0[:, -1] = 1

In [5]:
tableau0

array([[ 3.,  2.,  3.,  1.,  0.,  1.],
       [ 2.,  6.,  1.,  0.,  1.,  1.]])

In [6]:
# Player 1
tableau1 = np.empty((m, n+m+1))
tableau1[:, :m] = np.identity(m)
tableau1[:, m:m+n] = A
tableau1[:, -1] = 1

In [7]:
tableau1

array([[ 1.,  0.,  0.,  3.,  3.,  1.],
       [ 0.,  1.,  0.,  2.,  5.,  1.],
       [ 0.,  0.,  1.,  0.,  6.,  1.]])

Denote the player 0's variables by $x_0, x_1, x_2$ and $s_3, s_4$,
and the player 1's variables by $r_0, r_1, r_2$ and $y_3, y_4$.

The initial basic variables are $s_3, s_4$ and $r_0, r_1, r_2$.

In [8]:
basic_vars0 = np.arange(m, m+n)
basic_vars0

array([3, 4])

In [9]:
basic_vars1 = np.arange(0, m)
basic_vars1

array([0, 1, 2])

Let the initial pivot index be `1`,
so that $x_1$ is to enter the basis:

In [10]:
init_pivot = 1

In [11]:
# Current pivot
pivot = init_pivot

### Step 1

Determine the basic variable to leave by the minimum ratio test:

In [12]:
ratios = tableau0[:, -1] / tableau0[:, pivot]
ratios

array([ 0.5    ,  0.16667])

In [13]:
row_min = ratios.argmin()
row_min

1

In [14]:
basic_vars0[row_min]

4

$s_4$ is the basic variable that leaves the basis.

Update the tableau:

In [15]:
tableau0[row_min, :] /= tableau0[row_min, pivot]

In [16]:
tableau0

array([[ 3.     ,  2.     ,  3.     ,  1.     ,  0.     ,  1.     ],
       [ 0.33333,  1.     ,  0.16667,  0.     ,  0.16667,  0.16667]])

In [17]:
for i in range(tableau0.shape[0]):
    if i != row_min:
        tableau0[i, :] -= tableau0[row_min, :] * tableau0[i, pivot]

In [18]:
# Another approach by a NumPy trick
# ind = np.ones(tableau0.shape[0], dtype=bool)
# ind[row_min] = False
# tableau0[ind, :] -= tableau0[row_min, :] * tableau0[ind, :][:, [pivot]]

In [19]:
tableau0

array([[ 2.33333,  0.     ,  2.66667,  1.     , -0.33333,  0.66667],
       [ 0.33333,  1.     ,  0.16667,  0.     ,  0.16667,  0.16667]])

Update the basic variables and the pivot for the next step:

In [20]:
basic_vars0[row_min], pivot = pivot, basic_vars0[row_min]

In [21]:
basic_vars0

array([3, 1])

In [22]:
basic_vars0[row_min]

1

In [23]:
pivot

4

That is, $x_1$ has become a basic variable,
while $s_4$ becomes a nonbasic variable (i.e., $s_4 = 0$).

If the new pivot is equal to the initial pivot, we are done.

In [24]:
pivot == init_pivot

False

But this is `False`, so we continue.

In the next step, the variable $y_4$ which is *complementary* to $s_4$ (i.e., $y_4 s_4 = 0$)
becomes a basic variable.

### Step 2

Repeat the same exercise as above for `tableau1`.

In [25]:
tableau1

array([[ 1.,  0.,  0.,  3.,  3.,  1.],
       [ 0.,  1.,  0.,  2.,  5.,  1.],
       [ 0.,  0.,  1.,  0.,  6.,  1.]])

In [26]:
ratios = tableau1[:, -1] / tableau1[:, pivot]
row_min = ratios.argmin()
row_min

2

In [27]:
tableau1[row_min, :] /= tableau1[row_min, pivot]

In [28]:
tableau1

array([[ 1.     ,  0.     ,  0.     ,  3.     ,  3.     ,  1.     ],
       [ 0.     ,  1.     ,  0.     ,  2.     ,  5.     ,  1.     ],
       [ 0.     ,  0.     ,  0.16667,  0.     ,  1.     ,  0.16667]])

In [29]:
ind = np.ones(tableau1.shape[0], dtype=bool)
ind[row_min] = False
tableau1[ind, :] -= tableau1[row_min, :] * tableau1[ind, :][:, [pivot]]

In [30]:
tableau1

array([[ 1.     ,  0.     , -0.5    ,  3.     ,  0.     ,  0.5    ],
       [ 0.     ,  1.     , -0.83333,  2.     ,  0.     ,  0.16667],
       [ 0.     ,  0.     ,  0.16667,  0.     ,  1.     ,  0.16667]])

In [31]:
basic_vars1[row_min], pivot = pivot, basic_vars1[row_min]

In [32]:
pivot

2

In [33]:
basic_vars1

array([0, 1, 4])

In [34]:
basic_vars1[row_min]

4

That is, $y_4$ has become a basic variable,
while $r_2$ becomes a nonbasic variable.

If the new pivot is equal to the initial pivot, we are done.

In [35]:
pivot == init_pivot

False

But this is `False`, so we continue.

In the next step, the variable $x_2$ which is complementary to $r_2$
becomes a basic variable.

### Step 3

In [36]:
tableau0

array([[ 2.33333,  0.     ,  2.66667,  1.     , -0.33333,  0.66667],
       [ 0.33333,  1.     ,  0.16667,  0.     ,  0.16667,  0.16667]])

In [37]:
ratios = tableau0[:, -1] / tableau0[:, pivot]
row_min = ratios.argmin()
row_min

0

In [38]:
tableau0[row_min, :] /= tableau0[row_min, pivot]
ind = np.ones(tableau0.shape[0], dtype=bool)
ind[row_min] = False
tableau0[ind, :] -= tableau0[row_min, :] * tableau0[ind, :][:, [pivot]]

In [39]:
tableau0

array([[ 0.875 ,  0.    ,  1.    ,  0.375 , -0.125 ,  0.25  ],
       [ 0.1875,  1.    ,  0.    , -0.0625,  0.1875,  0.125 ]])

In [40]:
basic_vars0[row_min], pivot = pivot, basic_vars0[row_min]

In [41]:
pivot

3

In [42]:
pivot == init_pivot

False

### Step 4

In [43]:
tableau1

array([[ 1.     ,  0.     , -0.5    ,  3.     ,  0.     ,  0.5    ],
       [ 0.     ,  1.     , -0.83333,  2.     ,  0.     ,  0.16667],
       [ 0.     ,  0.     ,  0.16667,  0.     ,  1.     ,  0.16667]])

In [44]:
ratios = tableau1[:, -1] / tableau1[:, pivot]
row_min = ratios.argmin()
row_min

/usr/local/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


1

Note on the warning:

`tableau1[:, pivot]` has a zero entry, so we get a "divide by zero" warning.

In [45]:
tableau1[:, pivot]

array([ 3.,  2.,  0.])

We can just ignore it,
but we can also suppress it by
[`np.errstate`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.errstate.html)
with a `with` clause:

In [46]:
with np.errstate(divide='ignore'):
    ratios = tableau1[:, -1] / tableau1[:, pivot]

In [47]:
ratios

array([ 0.16667,  0.08333,      inf])

In [48]:
tableau1[row_min, :] /= tableau1[row_min, pivot]
ind = np.ones(tableau1.shape[0], dtype=bool)
ind[row_min] = False
tableau1[ind, :] -= tableau1[row_min, :] * tableau1[ind, :][:, [pivot]]

In [49]:
tableau1

array([[ 1.     , -1.5    ,  0.75   ,  0.     ,  0.     ,  0.25   ],
       [ 0.     ,  0.5    , -0.41667,  1.     ,  0.     ,  0.08333],
       [ 0.     ,  0.     ,  0.16667,  0.     ,  1.     ,  0.16667]])

In [50]:
basic_vars1[row_min], pivot = pivot, basic_vars1[row_min]

In [51]:
pivot

1

In [52]:
pivot == init_pivot

True

Now we have complete labeling, so we are done.

### Obtaining the Nash equilibrium

The basic variables are:

In [53]:
basic_vars0

array([2, 1])

$x_2$ and $x_1$, and

In [54]:
basic_vars1

array([0, 3, 4])

$r_0$, $y_3$, and $y_4$.

The indices of the basic variables corresponding to $x$:

In [55]:
basic_vars0[basic_vars0 < m]

array([2, 1])

The indices of the basic variables corresponding to $y$:

In [56]:
basic_vars1[basic_vars1 >= m]

array([3, 4])

The values of the basic variables are stored in the last columns of the tableaux.

The values of $x_2$ and $x_1$ are:

In [57]:
tableau0[basic_vars0 < m, -1]

array([ 0.25 ,  0.125])

The values of $y_3$ and $y_4$ are:

In [58]:
tableau1[basic_vars1 >= m, -1]

array([ 0.08333,  0.16667])

We need to normalize these values so that $x$ and $y$ are probability distributions.

In [59]:
x = np.zeros(m)
x[basic_vars0[basic_vars0 < m]] = tableau0[basic_vars0 < m, -1]
x /= x.sum()

In [60]:
x

array([ 0.     ,  0.33333,  0.66667])

In [61]:
y = np.zeros(n)
y[basic_vars1[basic_vars1 >= m] - m] = tableau1[basic_vars1 >= m, -1]
y /= y.sum()

In [62]:
y

array([ 0.33333,  0.66667])

The Nash equilibrium we have found is:

In [63]:
(x, y)

(array([ 0.     ,  0.33333,  0.66667]), array([ 0.33333,  0.66667]))

## Wrapping the procedure in a function

In [64]:
def pivoting(tableau, basic_vars, pivot):
    """
    Perform a complementary pivoting.
    
    Modify `tableau` and `basic_vars` in place and return the new pivot.
    
    """
    # Minimum ratio test
    with np.errstate(divide='ignore'):
        ratios = tableau[:, -1] / tableau[:, pivot]
    row_min = ratios.argmin()
    
    # Update the tableau
    tableau[row_min, :] /= tableau[row_min, pivot]
    ind = np.ones(tableau.shape[0], dtype=bool)
    ind[row_min] = False
    tableau[ind, :] -= tableau[row_min, :] * tableau[ind, :][:, [pivot]]
    
    # Update the basic variables and the pivot
    basic_vars[row_min], pivot = pivot, basic_vars[row_min]
    
    return pivot

Let us use the above function.

In [65]:
m, n = A.shape

tableau0 = np.empty((n, m+n+1))
tableau0[:, :m] = B_T
tableau0[:, m:m+n] = np.identity(n)
tableau0[:, -1] = 1

tableau1 = np.empty((m, n+m+1))
tableau1[:, :m] = np.identity(m)
tableau1[:, m:m+n] = A
tableau1[:, -1] = 1

tableaux = (tableau0, tableau1)

basic_vars0 = np.arange(m, m+n)
basic_vars1 = np.arange(0, m)

basic_vars = (basic_vars0, basic_vars1)

init_pivot = 1
players = [init_pivot // m, 1 - init_pivot // m]

pivot = init_pivot

while True:
    for i in players:
        pivot = pivoting(tableaux[i], basic_vars[i], pivot)
        if pivot == init_pivot:
            break
    else:
        continue
    break

out = np.zeros(m+n)
for i, (start, num) in enumerate(zip([0, m], [m, n])):
    ind = basic_vars[i] < start + num if i == 0 else start <= basic_vars[i]
    out[basic_vars[i][ind]] = tableaux[i][ind, -1]
    out[start:start+num] /= out[start:start+num].sum()

print("Nash equilibrium found\n", (out[:m], out[m:]))

Nash equilibrium found
 (array([ 0.     ,  0.33333,  0.66667]), array([ 0.33333,  0.66667]))


Note: There is no nested `break` in Python;
see e.g., [break two for loops](http://stackoverflow.com/questions/9038160/break-two-for-loops).